In [1]:
from google.colab import files
uploaded = files.upload()

Saving cryptonews.csv to cryptonews.csv


In [2]:
import pandas as pd
import tensorflow as tf

# Path ke file langsung
file_path = 'cryptonews.csv'

# Baca CSV dan ambil kolom title sebagai headline
headlines_df = pd.read_csv(file_path)
all_headlines = list(headlines_df.title.values)

print(f"Jumlah headline: {len(all_headlines)}")

Jumlah headline: 31037


In [3]:
print(all_headlines[:20])

['Grayscale CEO Calls for Simultaneous Approval of Spot Products to Level the Field', 'Indian Government is Actively Collaborating With Crypto Industry: Liminal Custody’s Country Head', 'Judge Approves Settlement: Binance to Pay $1.5 Billion to CFTC, CZ to Pay $150 Million Fine', 'Why a gold rush for inscriptions has broken half a dozen blockchains', "‘Concerning precedent’ — bloXroute Labs' MEV relays to reject OFAC blocks", 'Is This Why Bitcoin’s Price Rally Was Halted? (CryptoQuant)', 'Cathie Wood’s Ark Invest Sells $27.6 Million in GBTC Shares as Bitcoin Touches $43K Again', 'Bitcoin Soared 150% in 2023 But These Companies Did Even Better: Matrixport', 'South Korean City Busan Names Digital Exchange Preferred Bidder', 'SEC delays several Ethereum ETFs, pushing final decision to May', 'Zcash founder Zooko Wilcox steps down from Electric Coin, naming new CEO', 'Cipher Scales BTC Mining Operation with 37,000 Bitmain Antminers Worth $100M', 'South Korean Analysts: Global Crypto Market 

In [4]:
all_headlines[:20]

['Grayscale CEO Calls for Simultaneous Approval of Spot Products to Level the Field',
 'Indian Government is Actively Collaborating With Crypto Industry: Liminal Custody’s Country Head',
 'Judge Approves Settlement: Binance to Pay $1.5 Billion to CFTC, CZ to Pay $150 Million Fine',
 'Why a gold rush for inscriptions has broken half a dozen blockchains',
 "‘Concerning precedent’ — bloXroute Labs' MEV relays to reject OFAC blocks",
 'Is This Why Bitcoin’s Price Rally Was Halted? (CryptoQuant)',
 'Cathie Wood’s Ark Invest Sells $27.6 Million in GBTC Shares as Bitcoin Touches $43K Again',
 'Bitcoin Soared 150% in 2023 But These Companies Did Even Better: Matrixport',
 'South Korean City Busan Names Digital Exchange Preferred Bidder',
 'SEC delays several Ethereum ETFs, pushing final decision to May',
 'Zcash founder Zooko Wilcox steps down from Electric Coin, naming new CEO',
 'Cipher Scales BTC Mining Operation with 37,000 Bitmain Antminers Worth $100M',
 'South Korean Analysts: Global Cr

In [5]:
# Remove all headlines with the value of "Unknown"
all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

31037

In [6]:
all_headlines[:20]

['Grayscale CEO Calls for Simultaneous Approval of Spot Products to Level the Field',
 'Indian Government is Actively Collaborating With Crypto Industry: Liminal Custody’s Country Head',
 'Judge Approves Settlement: Binance to Pay $1.5 Billion to CFTC, CZ to Pay $150 Million Fine',
 'Why a gold rush for inscriptions has broken half a dozen blockchains',
 "‘Concerning precedent’ — bloXroute Labs' MEV relays to reject OFAC blocks",
 'Is This Why Bitcoin’s Price Rally Was Halted? (CryptoQuant)',
 'Cathie Wood’s Ark Invest Sells $27.6 Million in GBTC Shares as Bitcoin Touches $43K Again',
 'Bitcoin Soared 150% in 2023 But These Companies Did Even Better: Matrixport',
 'South Korean City Busan Names Digital Exchange Preferred Bidder',
 'SEC delays several Ethereum ETFs, pushing final decision to May',
 'Zcash founder Zooko Wilcox steps down from Electric Coin, naming new CEO',
 'Cipher Scales BTC Mining Operation with 37,000 Bitmain Antminers Worth $100M',
 'South Korean Analysts: Global Cr

In [7]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n',
    lower=True,
    split=' ',
    char_level=False,
    oov_token=None
)

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenize the words in our headlines
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_headlines)
total_words = len(tokenizer.word_index) + 1
print('Total words: ', total_words)

Total words:  22748


In [9]:
# Print a subset of the word_index dictionary created by Tokenizer
subset_dict = {key: value for key, value in tokenizer.word_index.items() \
               if key in ['a','man','a','plan','a','canal','panama']}
print(subset_dict)

{'a': 12, 'plan': 279, 'man': 1114, 'panama': 5495}


In [10]:
tokenizer.texts_to_sequences(['a','man','a','plan','a','canal','panama'])

[[12], [1114], [12], [279], [12], [], [5495]]

In [11]:
# Convert data to sequence of tokens
input_sequences = []
for line in all_headlines:
    # Convert our headline into a sequence of tokens
    token_list = tokenizer.texts_to_sequences([line])[0]

    # Create a series of sequences for each headline
    for i in range(1, len(token_list)):
        partial_sequence = token_list[:i+1]
        input_sequences.append(partial_sequence)

print(tokenizer.sequences_to_texts(input_sequences[:5]))
input_sequences[:5]

['grayscale ceo', 'grayscale ceo calls', 'grayscale ceo calls for', 'grayscale ceo calls for simultaneous', 'grayscale ceo calls for simultaneous approval']


[[332, 51],
 [332, 51, 287],
 [332, 51, 287, 7],
 [332, 51, 287, 7, 11671],
 [332, 51, 287, 7, 11671, 323]]

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Determine max sequence length
max_sequence_len = max([len(x) for x in input_sequences])

# Pad all sequences with zeros at the beginning to make them all max length
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0, 332,  51], dtype=int32)

In [13]:
# Predictors are every word except the last
predictors = input_sequences[:,:-1]
# Labels are the last word
labels = input_sequences[:,-1]
labels[:5]

array([   51,   287,     7, 11671,   323], dtype=int32)

In [14]:
from tensorflow.keras import utils

labels = utils.to_categorical(labels, num_classes=total_words)

In [15]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

# Input is max sequence length - 1, as we've removed the last word for the label
input_len = max_sequence_len - 1

model = Sequential()

# Add input embedding layer
model.add(Embedding(total_words, 10, input_length=input_len))

# Add LSTM layer with 100 units
model.add(LSTM(100))
model.add(Dropout(0.1))

# Add output layer
model.add(Dense(total_words, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [18]:
import numpy as np

dummy_predictors = np.random.rand(100, predictors.shape[1]).astype(np.float32)
dummy_labels = np.random.rand(100, labels.shape[1]).astype(np.float32)

model.fit(dummy_predictors, dummy_labels, epochs=3, batch_size=8)

Epoch 1/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 54ms/step - loss: 114072.0000
Epoch 2/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 114141.8359
Epoch 3/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - loss: 114148.9141


In [19]:
import psutil

ram_gb = psutil.virtual_memory().total / 1e9
print(f"Total RAM: {ram_gb:.2f} GB")

Total RAM: 13.61 GB


In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
import gc

# Bersihkan memori
K.clear_session()
gc.collect()

# Contoh data dummy
num_samples = 10000
num_features = 10
num_classes = 3

predictors = np.random.rand(num_samples, num_features).astype(np.float32)
labels = np.random.randint(0, num_classes, size=(num_samples,))
labels = tf.keras.utils.to_categorical(labels, num_classes=num_classes)

# Buat dataset tf.data
batch_size = 16
dataset = tf.data.Dataset.from_tensor_slices((predictors, labels))
dataset = dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Model sederhana
model = Sequential([
    Dense(64, activation='relu', input_shape=(num_features,)),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Training dengan dataset
model.fit(dataset, epochs=10, verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,883 (11.26 KB)

 Trainable params: 2,883 (11.26 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.3566 - loss: 1.0989
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3370 - loss: 1.0987
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3474 - loss: 1.0979
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3444 - loss: 1.0982
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3593 - loss: 1.0961
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.3560 - loss: 1.0964
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.3593 - loss: 1.0951
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.3619 - loss: 1.0952
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3638 - loss: 1.0944
Epoch 10/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3675 - loss: 1.0935


In [21]:
import numpy as np

def predict_next_token(seed_text):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list, verbose=0)
    predicted_class = np.argmax(predicted_probs, axis=-1)
    return predicted_class[0]  # hasil berupa 1 angka token ID

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Contoh data: kumpulan kalimat untuk tokenizer
texts = ["today in new york", "hello world", "machine learning is fun"]

# Buat tokenizer dan fit di data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# Ubah kalimat input jadi sequence token angka
sequence = tokenizer.texts_to_sequences(["today in new york"])

# Padding/truncating supaya panjang sequence jadi 10 (sesuai input model)
padded_seq = pad_sequences(sequence, maxlen=10, padding='pre')

print(padded_seq.shape)  # (1, 10)

# Lalu inputkan ke model
prediction = model.predict(padded_seq)
print(prediction)

(1, 10)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
[[0.521656   0.30037752 0.17796654]]


In [23]:
prediction = model.predict(padded_seq)  # shape (1, num_classes)
predicted_index = prediction.argmax(axis=-1)[0]  # ambil kelas dengan prediksi tertinggi
predicted_word = tokenizer.index_word.get(predicted_index, "unknown")
print(predicted_word)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
unknown


In [24]:
def generate_headline(seed_text, next_words=1):
    for _ in range(next_words):
        # Predict next token
        prediction = predict_next_token(seed_text)
        # Convert token to word
        next_word = tokenizer.sequences_to_texts([[prediction]])[0]

        # Add next word to the headline. This headline will be used in the next pass of the loop.
        seed_text += " " + next_word
    # Return headline as title-case
    return seed_text.title()

In [25]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def generate_headline(seed_text, next_words=5):
    for _ in range(next_words):
        # Tokenize seed_text jadi sequence angka
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        # Padding/truncating jadi panjang 10 sesuai model input
        token_list = pad_sequences([token_list], maxlen=10, padding='pre', truncating='pre')
        # Prediksi output (misal distribusi kelas kata)
        predicted_probs = model.predict(token_list, verbose=0)
        # Ambil kelas dengan probabilitas tertinggi
        predicted_index = np.argmax(predicted_probs, axis=-1)[0]
        # Cari kata dari tokenizer index_word
        output_word = tokenizer.index_word.get(predicted_index, "")
        # Tambah kata hasil prediksi ke seed_text
        seed_text += " " + output_word
    return seed_text

In [26]:
seed_texts = [
    'washington dc is',
    'today in new york',
    'the school district has',
    'crime has become']

for seed in seed_texts:
    print(generate_headline(seed, next_words=5))


washington dc is in    
today in new york     
the school district has today in in  
crime has become today in in  
